In [66]:
import pandas as pd
import numpy as np
import math
import os

# import for time, and file change
import json
from PIL import Image

# frequent updating
import time
import random

In [67]:
# define filepaths
workspace = "/Users/advaysingh/Documents/projects/hindi_classification/" 
#print("Current workspace:", workspace)

data = os.path.join(workspace, 'data/Hindi/')
dict_lib = os.path.join(workspace, 'data/dict.csv')
img_path = os.path.join(workspace, 'server/snapshot.png')
out_file = os.path.join(workspace, 'server/outputs.json')


In [70]:
# create train/test dicts with files and labels

def create_dict(x: str) -> dict:
    x_dict = {}
    i = 0
    for dir in os.listdir(os.path.join(data, x)):
        for file in os.listdir(os.path.join(data, x, dir)):
            x_dict[os.path.join(data, x, dir, file)] = i
        i += 1
    return x_dict

# make pandas df

train_df = pd.DataFrame.from_dict(create_dict('Train'), orient='index')
test_df = pd.DataFrame.from_dict(create_dict('Test'), orient='index')

#print(train_df)
#df_temp = train_df[0].drop_duplicates()
# print(df_temp)

In [269]:
# Activations class
class Activation:
    # can add more types
    def __init__(self, act_type: str) -> None:
        self.act = act_type

    def print_act(self) -> None:
        return self.act
    
    def compute(self, z) -> list:

        # Sigmoid function
        if (self.act == 'sigmoid'):
            n = []
            for val in np.array(z):
                n.append(1 / (1 + math.exp(val)))
            return n
        
        # ReLU function
        elif (self.act == 'relu'):
            return np.maximum(0, z)

    def prime(self, z: list):

        # Sigmoid prime
        if (self.act == 'sigmoid'):
            return (self.compute(z)) * (1 - self.compute(z))
        
        # ReLU prime
        if (self.act == 'relu'):
            return int(z > 0)


def softmax(costs: list) -> list:
    exp_vals = []
    for cost in np.array(costs):
        exp_vals.append(math.exp(cost))
    return_vals = []
    for i in range(len(exp_vals)):
        return_vals.append(exp_vals[i] / sum(exp_vals))
    return return_vals

def softmax_prime(costs: list) -> list:
    return softmax(costs) * (1 - softmax(costs))

n = softmax([2, 3, 4])
print(n)

[0.09003057317038046, 0.24472847105479767, 0.6652409557748219]


In [274]:

class Model:
    def __init__(self, train_df: pd.DataFrame,
                  act: Activation, layers: list, step: float, stochastic: bool, load_perams: bool) -> None:
        self.train_df = train_df

        # create activation function
        self.act = act
        self.layers = layers

        # init weight and bias with # layers np arrays
        # Using the Xavier weight init method https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/
        self.all_weights = [None] * (int(len(layers)))
        self.all_bias = [None] * (int(len(layers)))
        if (load_perams):
            layer = 0
            weightlist = os.listdir(os.path.join(workspace, 'data', 'Hyper_p/Weights'))
            for file in sorted(weightlist, key=lambda s: s.lower()):
                self.all_weights[layer] = pd.read_csv(os.path.join(workspace, 'data', 'Hyper_p/Weights', file)).to_numpy()
                #self.all_weights[layer] = np.delete(self.all_weights, [0], axis=1)
                print(self.all_weights[layer].shape)
                layer += 1
            layer = 0
            biaslist = os.listdir(os.path.join(workspace, 'data', 'Hyper_p/Biases'))
            for file in sorted(biaslist, key=lambda s: s.lower()):
                self.all_bias[layer] = pd.read_csv(os.path.join(workspace, 'data', 'Hyper_p/Biases', file)).to_numpy()
                layer += 1
            print(self.all_weights[0])
        else:
            for i in range(len(layers)):
                self.all_weights[i] = self.random_arrs(i)
                if i == 0:
                    self.all_bias[i] = np.random.uniform(low=-0.03125, high=0.03125, size=(self.layers[i],1))
                else:
                    bias_val = 1 / math.sqrt(self.layers[i - 1])
                    self.all_bias[i] = np.random.uniform(low=-bias_val, high=bias_val, size=(self.layers[i],1))
            #for val in self.all_weights:
               # print(val.shape)

        # init learn speed
        self.speed = step

        # determine gradient decent type
        self.stochastic = stochastic

    def random_arrs(self, layer: int):
        if (layer == 0):
            return(np.random.uniform(low=-0.03125, high=0.03125, size=(self.layers[layer],1024)))
        else:
            weight_val = 1 / math.sqrt(self.layers[layer - 1])
            return(np.random.uniform(low=-weight_val, high=weight_val, size=(self.layers[layer], self.layers[layer-1])))

    def img_to_np(self, dir) -> np.array:
        return np.array(Image.open(dir)).flatten()


    def print_weights(self) -> None:
        for arr in self.all_weights:
            print(arr.shape)

    def prop_forward(self, inputs: list, layer: int) -> list:
        a_vals = [0.0] * int(self.layers[layer])
        a_vals = np.dot(self.all_weights[layer], pd.DataFrame(inputs)) + self.all_bias[layer]
        if (layer == len(self.layers) - 1):
            return (softmax(a_vals))
        return(self.act.compute(a_vals))
    


    """def prop_backword(self, x: list, label: int, layer: int):
        outputs = self.prop_forward(x, layer)
        if (layer == len(self.layers) - 1):
            y = self.vals_for_softmax(label)
            for i in range(len(y) - 1):
                val = (y[i] - outputs[i]) * softmax_prime(outputs) 
                for j in range(len(x) - 1):
                    self.all_weights[layer][i][j] += val* x[j]
                    self.all_bias[layer][i][j] += val
                return(val)"""
    def prop_backward(self, x: list, label, list):
        delta_w = []
        delta_b = []
        for i in range(len(self.layers)):
            outs = self.prop_forward(x, i)


        for output in range(len(outputs)):
            val = float(self.prop_backword(outputs, label, layer + 1)) * float(sum(self.all_weights[layer][output]))
            for input in range(len(x) - 1):
                val += self.act.prime(x[input]) #edit
                self.all_weights[layer][output][input] += val * x[input]
                self.all_bias[layer][output][input] += val
            return val


    # potentially reconsider for time complexity
    def vals_for_softmax(self, index: int) -> list:
        vals = self.train_df[0].unique()
        for i in range(len(vals)):
            if vals[i] != index:
                vals[i] = 0
            else:
                vals[i] = 1
        return vals


    def train(self, epochs: int) -> None:
        # call back prop with dataset epochs times
        for epoch in  range(epochs):
            for img_dir, row in self.train_df.iterrows():
                delta_w, delta_b += self.prop_backword(self.img_to_np(img_dir).tolist(), row[0])
            delta_w, delta_b = delta_w / self.train_df.size, delta_b / self.train_df.size
            self.all_weights += delta_w
            self.all_bias += delta_b
            print("epoch: ", epoch, self.validate(train_df))

    def pred(self, img: np.array) -> int:
        for i in range(len(self.layers)):
            img = self.prop_forward(img, i)
        return img

    def validate(self, test_df: pd.DataFrame) -> None:
        count, countright = 0, 0
        for img_dir, row in test_df.iterrows():
            outputs = self.pred(self.img_to_np(img_dir))
            actual = list(self.vals_for_softmax(row[0]))
            if (outputs.index(max(outputs))) == (actual.index(max(actual))):
                countright += 1
            count += 1
        print("count: ", count, "countright: ", countright)

    def save(self) -> None:
        hp_data = '/Users/advaysingh/Documents/projects/hindi_classification/data/hyper_p'
        hp_data_biases, hp_data_weights = hp_data + '/Biases', hp_data + '/Weights'
        os.makedirs(hp_data)
        os.makedirs(hp_data_biases)
        os.makedirs(hp_data_weights)
        for layer in range(len(self.layers)):
            df_weights = pd.DataFrame(self.all_weights[layer])
            df_bias = pd.DataFrame(self.all_bias[layer])
            weights_file_name = "layer_" + str(layer) + "weights.csv"
            bias_file_name = "layer_" + str(layer) + "bias.csv"
            weights_file = open(os.path.join(hp_data_weights, weights_file_name), "x")
            bias_file = open(os.path.join(hp_data_biases, bias_file_name), "x")
            df_weights.to_csv(weights_file, index=False)
            df_bias.to_csv(bias_file, index=False)


    """def print_weights(self, layer: int) -> None:
        for row in self.all_weights[layer]:
            for weight in row:
                print(weight)"""



In [275]:
cnn = Model(train_df, Activation('sigmoid'), [5, 46], 1.2, False, False)
cnn.print_weights()
#cnn.prop_forward([3] * 1024, 0)
#cnn.save()
#cnn.print_weights(1)
#cnn.validate(test_df)
#cnn.train(1) # something for committing

(5, 1024)
(46, 5)


In [276]:

cnn.validate(test_df)

/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/60257770.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n.append(1 / (1 + math.exp(val)))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/60257770.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  exp_vals.append(math.exp(cost))


count:  13800 countright:  339


In [277]:
cnn.train(1)

/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/60257770.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n.append(1 / (1 + math.exp(val)))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/60257770.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  exp_vals.append(math.exp(cost))


TypeError: unsupported operand type(s) for -: 'int' and 'list'

In [265]:
import json
import time
import random

from PIL import Image

cnn = Model(train_df, Activation('sigmoid'), [5, 46], 1.2, False, True)
"""outs = cnn.pred(cnn.img_to_np(img_path))
index = outs.index(max(outs))
new_english_value = pd.read_csv(os.path.join(workspace, 'data', 'dict.csv'))[index][1]"""

def update_json_file(hindi_value, english_value):
    with open(out_file, 'r') as file:
        data = json.load(file)
    data['hindi'] = hindi_value
    data['english'] = english_value
    with open(out_file, 'w') as file:
        json.dump(data, file, indent=4)

# Define a list of possible values if applicable
hindi_values = ["हिं", "नया", "मूल्य", "उदाहरण"]
english_values = ["Eng", "New", "Value", "Example"]

# Loop to update the JSON file every second with a random value
try:
    while True:
        # Generate a random value from the lists
        new_hindi_value = random.choice(hindi_values)
        #new_english_value = random.choice(english_values)
        img = np.array(Image.open(img_path).convert("L").resize((32, 32))).flatten()
        index = cnn.pred(img).index(max(cnn.pred(img)))
        new_english_value = str(pd.read_csv(os.path.join(workspace, 'data', 'dict.csv')).iloc[index, 1])

        update_json_file(new_hindi_value, new_english_value)
        
        # Print the new values (for debug purposes)
        # print(f'Updated JSON file with hindi: {new_hindi_value}, english: {new_english_value}')

        # read in and resize img
 
        # read in image for debugging
        """img.show("img to classify")
        print(img.format)
        print(img.mode)
        print(img.size)"""

        # create np array and resize
        """img_data = np.array(img).flatten()
        print(img_data.shape)
        print(img_data)"""

        
        time.sleep(10)  # Wait for 1 second
except KeyboardInterrupt:
    print("Update stopped by user.")

(5, 1024)
(46, 5)
[[ 0.00516757  0.00772424 -0.02965045 ...  0.01455595 -0.0021856
   0.00281147]
 [ 0.02902657  0.02789728  0.02882575 ... -0.02899435 -0.00561021
   0.00619948]
 [-0.01282714  0.02266769  0.01177728 ... -0.01740452 -0.01306084
  -0.0256642 ]
 [-0.0306379  -0.01406958 -0.02793074 ... -0.02759458  0.01153612
   0.01584119]
 [-0.00979872  0.00293341  0.02098712 ...  0.02134781  0.02585534
   0.01623596]]
0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.960771318722247
1.6521326812058794
1.443654924154537
1.458406310146429
0.87356049487839
0.8588027259728629
0.5644177581927523
0.9755047655816174
0.9540986693477378
0.8462086032801243
1.0946882716864994
0.7228554216659109
0.9759989643074628
0.7748936645843225
1.6248730758407122
0.5899087023518892
0.9299912747169965
1.949537472741643
1.2390114272642143
1.8096887207345569
1.5932907173114823
0.8170918608647918
1.011802138127133
0.430803966753

/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n.append(1 / (1 + math.exp(val)))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(math.exp(cost))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  exp_vals.append(math.exp(cost

0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.960771318722247
1.6521326812058794
1.443654924154537
1.458406310146429
0.87356049487839
0.8588027259728629
0.5644177581927523
0.9755047655816174
0.9540986693477378
0.8462086032801243
1.0946882716864994
0.7228554216659109
0.9759989643074628
0.7748936645843225
1.6248730758407122
0.5899087023518892
0.9299912747169965
1.949537472741643
1.2390114272642143
1.8096887207345569
1.5932907173114823
0.8170918608647918
1.011802138127133
0.43080396675348437
0.8273173066667576
1.308141831858087
1.292653607875661
1.3112828184033525
1.0399561546921257
0.7854417190091054
1.917537009028405
2.040899951689852
0.9307867026255214
1.0377776334272317
1.204820928043534
1.2056641493319384
0.7610830460716267
1.5047985405093014
1.1851686599403946
0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.96

/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n.append(1 / (1 + math.exp(val)))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(math.exp(cost))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  exp_vals.append(math.exp(cost

0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.960771318722247
1.6521326812058794
1.443654924154537
1.458406310146429
0.87356049487839
0.8588027259728629
0.5644177581927523
0.9755047655816174
0.9540986693477378
0.8462086032801243
1.0946882716864994
0.7228554216659109
0.9759989643074628
0.7748936645843225
1.6248730758407122
0.5899087023518892
0.9299912747169965
1.949537472741643
1.2390114272642143
1.8096887207345569
1.5932907173114823
0.8170918608647918
1.011802138127133
0.43080396675348437
0.8273173066667576
1.308141831858087
1.292653607875661
1.3112828184033525
1.0399561546921257
0.7854417190091054
1.917537009028405
2.040899951689852
0.9307867026255214
1.0377776334272317
1.204820928043534
1.2056641493319384
0.7610830460716267
1.5047985405093014
1.1851686599403946
0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.96

...

0.6899322665875139
1.0370746694493629
1.0052859108288612
1.9917375969403615
1.2645306841814525
1.541279345432969
0.6403918065234844
0.8969307191768289
1.5819200714070996
1.5993856356923208
1.8909411790653765
1.6605484459734834
0.9834222577151077
0.45377726980469396
0.8297986980400459
0.9080995904486122
0.7334271226965837
1.3794376170061031
0.729640463905336
1.0103423257806965
1.003982101081032
1.8166511080705845
0.6144630214103457
1.086399189532544
1.5418959052356513
1.0478112831768283
2.503164504675944
1.2795588520215593
0.7990647291468177
1.4199755737033168
0.5028664550647873
0.8957954247878321
2.159794003737083
1.334464910931545
1.5692470015421593
1.4463090078417449
0.7691663736091849
1.6836760166898759
2.6048333264607737
1.0801467941250489
1.1164106758694574
1.184798719975711
1.3693790711685767
0.4454651735825002
1.55554758884221
1.0258630741720924
0.6899322665875139
1.0370746694493629
1.0052859108288612
1.9917375969403615
1.2645306841814525
1.541279345432969
0.6403918065234844
0.8

/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n.append(1 / (1 + math.exp(val)))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(math.exp(cost))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  exp_vals.append(math.exp(cost

0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.960771318722247
1.6521326812058794
1.443654924154537
1.458406310146429
0.87356049487839
0.8588027259728629
0.5644177581927523
0.9755047655816174
0.9540986693477378
0.8462086032801243
1.0946882716864994
0.7228554216659109
0.9759989643074628
0.7748936645843225
1.6248730758407122
0.5899087023518892
0.9299912747169965
1.949537472741643
1.2390114272642143
1.8096887207345569
1.5932907173114823
0.8170918608647918
1.011802138127133
0.43080396675348437
0.8273173066667576
1.308141831858087
1.292653607875661
1.3112828184033525
1.0399561546921257
0.7854417190091054
1.917537009028405
2.040899951689852
0.9307867026255214
1.0377776334272317
1.204820928043534
1.2056641493319384
0.7610830460716267
1.5047985405093014
1.1851686599403946
0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.96

/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n.append(1 / (1 + math.exp(val)))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(math.exp(cost))
/var/folders/kd/c2gt634j3jd1_cgnqpmpswhw0000gn/T/ipykernel_81072/950884148.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  exp_vals.append(math.exp(cost

0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.960771318722247
1.6521326812058794
1.443654924154537
1.458406310146429
0.87356049487839
0.8588027259728629
0.5644177581927523
0.9755047655816174
0.9540986693477378
0.8462086032801243
1.0946882716864994
0.7228554216659109
0.9759989643074628
0.7748936645843225
1.6248730758407122
0.5899087023518892
0.9299912747169965
1.949537472741643
1.2390114272642143
1.8096887207345569
1.5932907173114823
0.8170918608647918
1.011802138127133
0.43080396675348437
0.8273173066667576
1.308141831858087
1.292653607875661
1.3112828184033525
1.0399561546921257
0.7854417190091054
1.917537009028405
2.040899951689852
0.9307867026255214
1.0377776334272317
1.204820928043534
1.2056641493319384
0.7610830460716267
1.5047985405093014
1.1851686599403946
0.6442762304242494
0.8886873358049505
1.1504389981168142
2.5547491467422634
1.8244719919331795
1.9537041768184336
0.8354939514239575
0.96